### Summer Days: wps_climdex_su

WPS wrapper for [climdex.pcic](https://cran.r-project.org/web/packages/climdex.pcic/climdex.pcic.pdf) function, [climdex.su](https://cran.r-project.org/web/packages/climdex.pcic/climdex.pcic.pdf#page=25)

Takes a climdexInput object and computes the SU (summer days) climdexindex:  that is,  the annual count of days where daily maximum temperature exceeds 25 degreesCelsius.

In [1]:
import os
import requests
from birdy import WPSClient
from rpy2 import robjects
from urllib.request import urlretrieve
from pkg_resources import resource_filename
from tempfile import NamedTemporaryFile

from wps_tools.testing import get_target_url, local_path

In [2]:
# Ensure we are in the working directory with access to the data
while os.path.basename(os.getcwd()) != "quail":
    os.chdir('../')

In [3]:
# NBVAL_IGNORE_OUTPUT
url = get_target_url("chickadee")
print(f"Using chickadee on {url}")
url="http://localhost:5005/wps"

Using chickadee on https://docker-dev03.pcic.uvic.ca/twitcher/ows/proxy/chickadee/wps


In [4]:
quail = WPSClient(url)

#### Help for individual processes can be diplayed using the ? command (ex/ bird.process?)

In [5]:
# NBVAL_IGNORE_OUTPUT
quail.climdex_su?

Signature: quail.climdex_su(climdex_input, output_path=None, loglevel='INFO')
Docstring:
Computes the annual count of days where daily maximum temperature exceeds 25 degreesCelsius

Parameters
----------
climdex_input : ComplexData:mimetype:`application/rds`
    R Object of type climdexInput
output_path : string
    Filename to store the count of days where tmax > 25 degC for each year (extension .rda)
loglevel : {'CRITICAL', 'ERROR', 'WARNING', 'INFO', 'DEBUG', 'NOTSET'}string
    Logging level

Returns
-------
summer_days_file : ComplexData:mimetype:`application/rds`
    A vector containing the number of summer days for each year
File:      ~/code/birds/quail/</tmp/quail-venv/lib/python3.8/site-packages/birdy/client/base.py-0>
Type:      method


#### Run wps_climdex_su process

In [6]:
with NamedTemporaryFile(suffix=".rda", prefix="summer_days_", dir="/tmp", delete=True) as output_file:
    output = quail.climdex_su(
            climdex_input=resource_filename("tests","data/climdexInput.rds"),
            output_path=output_file.name
        )

In [7]:
url = output.get()[0]

#### Test output against expected output

In [17]:
with NamedTemporaryFile(suffix=".rda", prefix="tmp_copy", dir="/tmp", delete=True) as tmp_file:
    urlretrieve(url, tmp_file.name)
    robjects.r("load(file='{}')".format(tmp_file.name))
    output_vector = robjects.r("summer_days")
    
robjects.r("load(file='{}')".format(resource_filename("tests","data/expected_summer_days.rda")))
expected_vector = robjects.r("expected_summer_days")
    
for index in range(len(expected_vector)):
    assert str(output_vector[index]) == str(expected_vector[index])